- Connect objects in a geographic table to one another using the spatial relationships between them. 

- Spatial weights matrices express topology(mathematical structure that expresses the connectivity between observations), letting us embed all of our observations in space together, rather than asking and answering single questions about features nearby a unit.

In [ ]:
import contextily
import geopandas
import rioxarray
import seaborn
import pandas
import numpy
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from pysal.lib import cg as geometry

from pysal.lib import weights

import warnings

warnings.filterwarnings("ignore")

### Contiguity Weights

- Those who share a common border

In [ ]:
# Get points in a grid

l = numpy.arange(3)
xs, ys = numpy.meshgrid(l, l)

# Generate polygons
polys = []
for x, y in zip(xs.flatten(), ys.flatten()):
    poly = Polygon([(x,y), (x+1,y), (x+1, y+1), (x, y+1)])
    polys.append(poly)
#Convert to Geoseries    
polys =geopandas.GeoSeries(polys)
gdf = geopandas.GeoDataFrame( {
        "geometry": polys,
        "id": ["P-%s" % str(i).zfill(2) for i in range(len(polys))],
    }
)
    

In [ ]:
# Plot grid geotable
ax = gdf.plot(facecolor="w", edgecolor="k")

# Loop over each cell and add the text
for x, y, t in zip(
    [p.centroid.x - 0.25 for p in polys],
    [p.centroid.y - 0.25 for p in polys],
    [i for i in gdf["id"]],
):
    plt.text(
        x,
        y,
        t,
        verticalalignment="center",
        horizontalalignment="center",
    )

# Remove axes
ax.set_axis_off()
plt.show()



In [ ]:
# Build a rook contiguity matrix from a regular 3x3
# lattice stored in a geo-table
wr = weights.contiguity.Rook.from_dataframe(gdf)

In [ ]:
wr.neighbors

In [ ]:
pandas.DataFrame(*wr.full()).astype(int)

In [ ]:
# Build a queen contiguity matrix from a regular 3x3
# lattice stored in a geo-table
wq = weights.contiguity.Queen.from_dataframe(gdf)
wq.neighbors